# Exercises

In [156]:

import pyspark
import numpy as np
import pandas as pd
import pydataset

spark = pyspark.sql.SparkSession.builder.getOrCreate()
# Note: The pyspark avg and mean functions are aliases of eachother
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import lit
from pyspark.sql.functions import *

### 1.Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [150]:
#create a dictionary
pl = {
    'language': ['python', 'sql', 'c++', 'java', 'pascal']
    
}

In [151]:
#create a pandas df
pandas_dataframe = pd.DataFrame(pl)
pandas_dataframe

,language
0,python
1,sql
2,c++
3,java
4,pascal


In [153]:
#other way
pl = pd.DataFrame(
    {"language": ["r", "python", "sql", "julia", "pig latin", "lorem ipsum"]}
)
df = spark.createDataFrame(pl)

In [154]:
#create aspark df
df = spark.createDataFrame(pandas_dataframe)
df

DataFrame[language: string]

In [24]:
#schema
df.printSchema()

root
 |-- language: string (nullable = true)



In [17]:
#shape
print((df.count(), len(df.columns)))

(5, 1)


In [18]:
# check the first 5 rows
df.show(5)

+--------+
|language|
+--------+
|  python|
|     sql|
|     c++|
|    java|
|  pascal|
+--------+



###  2. Load the mpg dataset as a spark dataframe.

In [19]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



    a. Create 1 column of output that contains a message like the one below:


    ```The 1999 audi a4 has a 4 cylinder engine.```

    For each vehicle.

In [147]:
mpg.select('*', concat(lit("The "),mpg.year, mpg.manufacturer, lit(" "), mpg.model,
       lit(" has a  "), mpg.cyl, lit(" cylinder engine" )).alias('description')).show(truncate=False)

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-------------------------------------------------------------+
|manufacturer|model             |displ|year|cyl|trans     |drv|cty|hwy|fl |class  |description                                                  |
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+-------------------------------------------------------------+
|audi        |a4                |1.8  |1999|4  |auto(l5)  |f  |18 |29 |p  |compact|The 1999audi a4 has a  4 cylinder engine                     |
|audi        |a4                |1.8  |1999|4  |manual(m5)|f  |21 |29 |p  |compact|The 1999audi a4 has a  4 cylinder engine                     |
|audi        |a4                |2.0  |2008|4  |manual(m6)|f  |20 |31 |p  |compact|The 2008audi a4 has a  4 cylinder engine                     |
|audi        |a4                |2.0  |2008|4  |auto(av)  |f  |21 |30 |p  |compact|The 2008audi a4 has a  4 cylinder engine 

In [52]:
#here I'm just geting one column in the df
new_df = mpg.select('*', concat(lit("The "),mpg.year, mpg.manufacturer, lit(" "), mpg.model,
       lit(" has a  "), mpg.cyl, lit(" cylinder engine" )).alias('description'))

[Row(description='The 1999audi a4 has a  4 cylinder engine'),
 Row(description='The 1999audi a4 has a  4 cylinder engine'),
 Row(description='The 2008audi a4 has a  4 cylinder engine'),
 Row(description='The 2008audi a4 has a  4 cylinder engine'),
 Row(description='The 1999audi a4 has a  6 cylinder engine')]

In [51]:
#this is the new column
new_col = concat(lit("The "),mpg.year, mpg.manufacturer, lit(" "), mpg.model,
        lit(" has a  "), mpg.cyl, lit(" cylinder engine" ))

In [55]:
#new df with the existing 
new_mpg =mpg.select('*', new_col.alias('description'))

In [56]:
new_mpg.show()

+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------+
|manufacturer|             model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|         description|
+------------+------------------+-----+----+---+----------+---+---+---+---+-------+--------------------+
|        audi|                a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|The 1999audi a4 h...|
|        audi|                a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|The 1999audi a4 h...|
|        audi|                a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|The 2008audi a4 h...|
|        audi|                a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|The 2008audi a4 h...|
|        audi|                a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|The 1999audi a4 h...|
|        audi|                a4|  2.8|1999|  6|manual(m5)|  f| 18| 26|  p|compact|The 1999audi a4 h...|
|        audi|                a4|  3.1|2008|  6|  auto(

    b. Transform the trans column so that it only contains either manual or auto.

In [101]:
from pyspark.sql.functions import expr
from pyspark.sql.functions import when

In [102]:
mpg.select(mpg.trans, when((mpg.trans == 'manual(m5)') | (mpg.trans == 'manual(m6)'), 'manual').otherwise("auto")).show()

+----------+----------------------------------------------------------------------------------+
|     trans|CASE WHEN ((trans = manual(m5)) OR (trans = manual(m6))) THEN manual ELSE auto END|
+----------+----------------------------------------------------------------------------------+
|  auto(l5)|                                                                              auto|
|manual(m5)|                                                                            manual|
|manual(m6)|                                                                            manual|
|  auto(av)|                                                                              auto|
|  auto(l5)|                                                                              auto|
|manual(m5)|                                                                            manual|
|  auto(av)|                                                                              auto|
|manual(m5)|                            

In [157]:
# multiple ways to do it 
mpg.select(
    'trans',
    regexp_extract("trans", r"^(\w+)\(", 1).alias("regexp_extract"),
    regexp_replace("trans", r"\(.+$", "").alias("regexp_replace"),
    when(
        mpg.trans.like("auto%"), "auto"
    ).otherwise("manual").alias("when + like")
).show()

+----------+--------------+--------------+-----------+
|     trans|regexp_extract|regexp_replace|when + like|
+----------+--------------+--------------+-----------+
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|manual(m6)|        manual|        manual|     manual|
|  auto(av)|          auto|          auto|       auto|
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(av)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(l5)|          auto|          auto|       auto|
|manual(m6)|        manual|        manual|     manual|
|  auto(s6)|          auto|          auto|       auto|
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(s6)|          auto|          auto|       auto|
|manual(m6)|        manual|        manual|     manual|
|  auto(l5

3. Load the tips dataset as a spark dataframe.

In [110]:
tips = pydataset.data('tips')

In [111]:
tips = spark.createDataFrame(tips)
tips.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

    a. What percentage of observations are smokers?

In [119]:
# number of smokers
smokers = tips.where(tips.smoker == 'Yes').count()

In [137]:
smokers

93

In [120]:
#number of customers
total = tips.count()

In [148]:
total

244

In [123]:
round(smokers / total * 100 , 2)

38.11

In [125]:
print(f' The percentage of customers that are smokers is : {round(smokers / total * 100 , 2)} %')

 The percentage of customers that are smokers is : 38.11 %


In [ ]:
#other way to do it

In [158]:

tips.groupBy("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [159]:
tips.groupBy("smoker").count().withColumn(
    "percent",
    concat(round((col("count") / tips.count() * 100), 0).cast("int"), lit("%")),
).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



    b. Create a column that contains the tip percentage

In [130]:
col = (tips.tip / tips.total_bill)
col

Column<'(tip / total_bill)'>

In [132]:
tips.select(col).show(5)

+-------------------+
| (tip / total_bill)|
+-------------------+
|0.05944673337257211|
|0.16054158607350097|
|0.16658733936220846|
| 0.1397804054054054|
|0.14680764538430255|
+-------------------+
only showing top 5 rows



In [133]:
tips_pct = tips.select('*', col.alias('tip_pct'))

In [134]:
tips_pct.show(5)

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|            tip_pct|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
+----------+----+------+------+---+------+----+-------------------+
only showing top 5 rows



In [160]:
#other way
tips.withColumn("tip_percentage", col('tip') / col('total_bill')).show()

+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|0.18623962040332148|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|0.22805017103762829|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|0.11607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|0.13031914893617022|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| 0.2185385656292287|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 0.1665043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|0

    c. Calculate the average tip percentage for each combination of sex and smoker.

In [136]:
tips_pct.groupBy('sex', 'smoker').agg(mean('tip_pct')).show()

+------+------+-------------------+
|   sex|smoker|       avg(tip_pct)|
+------+------+-------------------+
|  Male|    No| 0.1606687151291298|
|  Male|   Yes| 0.1527711752024851|
|Female|    No| 0.1569209707691836|
|Female|   Yes|0.18215035269941035|
+------+------+-------------------+



In [161]:
#other way to do it
(
    tips.withColumn("tip_percentage", col('tip') / col('total_bill'))
    .groupby("sex")
    .pivot("smoker")
    .agg(round(mean("tip_percentage"), 4))
    .show()
)

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [162]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



- Convert the temperatures to fahrenheit. formula (0°C × 9/5) + 32

In [139]:
temp_maxC= (weather.temp_max * 9/5) + 32

Column<'(((temp_max * 9) / 5) + 31)'>

In [145]:
tem_minC = (weather.temp_min * 9/5) + 32

+---------------------------+
|(((temp_min * 9) / 5) + 31)|
+---------------------------+
|                       40.0|
|                      36.04|
|                      43.96|
|                      41.08|
|                      36.04|
|                      34.96|
|                      36.04|
|                      36.04|
|                       40.0|
|                      32.08|
|                      29.02|
|                      27.94|
|                      25.96|
|                      32.08|
|         25.060000000000002|
|                      25.96|
|                       31.0|
|                      25.96|
|                      25.96|
|                      29.02|
+---------------------------+
only showing top 20 rows



In [163]:
weather = weather.withColumn(
    "temp_max", (col("temp_max") *9 /5 +32)
)

In [166]:
weather = weather.withColumn(
    "temp_min", (col("temp_min") *9 /5 +32)
)

In [167]:
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|   55.04|    41.0| 4.7|drizzle|
|2012-01-02|         10.9|   51.08|   37.04| 4.5|   rain|
|2012-01-03|          0.8|   53.06|   44.96| 2.3|   rain|
|2012-01-04|         20.3|   53.96|   42.08| 4.7|   rain|
|2012-01-05|          1.3|   48.02|   37.04| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [168]:
#other way to do it
# pandas equivalent -- df.temp_max = df.temp_max * 9 / 5 + 32

weather = weather.withColumn(
    "temp_max", (col("temp_max") * 9 / 5 + 32)
).withColumn("temp_min", (col("temp_min") * 9 / 5 + 32))

In [169]:
weather.show(4)

+----------+-------------+------------------+--------+----+-------+
|      date|precipitation|          temp_max|temp_min|wind|weather|
+----------+-------------+------------------+--------+----+-------+
|2012-01-01|          0.0|           131.072|   105.8| 4.7|drizzle|
|2012-01-02|         10.9|123.94399999999999|  98.672| 4.5|   rain|
|2012-01-03|          0.8|127.50800000000001| 112.928| 2.3|   rain|
|2012-01-04|         20.3|           129.128| 107.744| 4.7|   rain|
+----------+-------------+------------------+--------+----+-------+
only showing top 4 rows



- Which month has the most rain, on average?

In [179]:
(
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_rainfall"))
    .sort("total_rainfall")
    .show()
)

+-----+----+------------------+
|month|year|    total_rainfall|
+-----+----+------------------+
|    8|2012|               0.0|
|    7|2013|               0.0|
|    9|2012|0.8999999999999999|
|    7|2015|               2.3|
|    6|2015|5.8999999999999995|
|    5|2015|14.799999999999999|
|    6|2014|18.800000000000004|
|    7|2014|              19.6|
|    9|2015|              21.1|
|    7|2012|              26.3|
|    6|2013|              33.1|
|    8|2013|              34.4|
|   10|2013|39.199999999999996|
|    2|2013|40.300000000000004|
|   12|2013| 42.39999999999999|
|    8|2014| 45.99999999999999|
|    4|2015| 51.59999999999999|
|    5|2012|52.199999999999996|
|    9|2014|56.699999999999996|
|    5|2013| 60.49999999999999|
+-----+----+------------------+
only showing top 20 rows



In [182]:
 (
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
).show(5)


+-----+------------------+
|month|  avg_monthly_rain|
+-----+------------------+
|   11|           160.625|
|   12|           155.675|
|    3|            151.55|
|   10|            125.85|
|    1|116.49999999999997|
+-----+------------------+
only showing top 5 rows



In [183]:
row = (
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
    .first()
)
row

Row(month=11, avg_monthly_rain=160.625)

In [187]:
row?

- Which year was the windiest?

In [186]:
(
weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind")).show(5)
)

+----+------------------+
|year|     total of wind|
+----+------------------+
|2015|            1153.3|
|2013|1100.8000000000002|
|2014|1236.5000000000005|
|2012|1244.6999999999998|
+----+------------------+



In [188]:
(
weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_wind")).show(5)
)

+----+------------------+
|year|        total_wind|
+----+------------------+
|2015|            1153.3|
|2013|1100.8000000000002|
|2014|1236.5000000000005|
|2012|1244.6999999999998|
+----+------------------+



- What is the most frequent type of weather in January?

- What is the average high and low temperature on sunny days in July in 2013 and 2014?

- What percentage of days were rainy in q3 of 2015?

- For each year, find what percentage of days it rained (had non-zero precipitation).